In [ ]:
from CountryCountsByBatchcodeTable2BarChartDescriptionTableConverter import CountryCountsByBatchcodeTable2BarChartDescriptionTableConverter
from JensenShannonDistance2BarChartDescriptionColumnAdder import JensenShannonDistance2BarChartDescriptionColumnAdder
from CountriesByBatchcodeProvider import getCountryCountsByBatchcodeTable, filterByBatchcodes
from Column2DataframeAdder import addColumn2Dataframe
from BatchCodeTableFactory import BatchCodeTableFactory
from HistogramFactoryAndPersister import createAndSaveGlobalHistograms
from SymptomByBatchcodeTableFactory import SymptomByBatchcodeTableFactory
from BatchCodeTablePersister import createGlobalBatchCodeTable
from BatchCodeTableHtmlUpdater import updateBatchCodeTableHtmlFile, saveLastUpdated2HtmlFile
from InternationalVaersCovid19Provider import getInternationalVaersCovid19, getInternationalVaersCovid19BeforeDeletion, get_international_VAERSVAX_VAERSSYMPTOMS_Covid19
from DateProvider import DateProvider
from BarChartDescriptionTable2DictionaryConverter import BarChartDescriptionTable2DictionaryConverter
from CountryColumnsMerger import CountryColumnsMerger
from datetime import datetime
from VAERSFileDownloader import updateVAERSFiles
from GoogleAnalytics.CityCountsByBatchcodeTablesMerger import CityCountsByBatchcodeTablesMerger
from IOUtils import IOUtils
from pathlib import Path
from GoogleAnalytics.Helper import persistCityCountsByClickedBatchcodeTables
import os
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
# split --bytes=45MiB VAERSBeforeDeletion.7z VAERSBeforeDeletion_
def restoreVAERSBeforeDeletionFolder():
    os.system('mkdir -p VAERS')
    os.system('cat data/VAERSBeforeDeletion/VAERSBeforeDeletion_* > VAERS/VAERSBeforeDeletion.7z')
    os.system('cd VAERS; 7z x -y VAERSBeforeDeletion.7z')

vaersBeforeDeletionDataDir = 'VAERS/VAERSBeforeDeletion'
restoreVAERSBeforeDeletionFolder()

In [ ]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())
print('needsUpdate:', dateProvider.needsUpdate())


In [ ]:
years_from_start_of_COVID_vaccination_to_present = list(range(2020, datetime.now().year + 1))


In [ ]:
updateVAERSFiles(
        years = years_from_start_of_COVID_vaccination_to_present,
        workingDirectory = os.getcwd())

In [ ]:
international_VAERSVAX_Covid19, international_VAERSSYMPTOMS = get_international_VAERSVAX_VAERSSYMPTOMS_Covid19(years = years_from_start_of_COVID_vaccination_to_present)

In [ ]:
international_VAERSVAX_Covid19


In [ ]:
international_VAERSSYMPTOMS


In [ ]:
internationalVaersCovid19 = getInternationalVaersCovid19(dataDir = 'VAERS', years = years_from_start_of_COVID_vaccination_to_present)
internationalVaersCovid19

In [ ]:
internationalVaersCovid19 = CountryColumnsMerger.mergeCountryColumnOfSrcIntoDst(
    src = getInternationalVaersCovid19BeforeDeletion(vaersBeforeDeletionDataDir),
    dst = internationalVaersCovid19)
internationalVaersCovid19

In [ ]:
batchCodeTable = createGlobalBatchCodeTable(
    deleteEntriesWithADRsLessThanOrEqual = 2,
    minADRsForLethality = 100,
    batchCodeTableFactory = BatchCodeTableFactory(internationalVaersCovid19))
batchCodeTable

In [ ]:
IOUtils.saveDataFrameAsJson(batchCodeTable, '../docs/data/batchCodeTables/Global.json')
saveLastUpdated2HtmlFile(
    lastUpdated = dateProvider.getLastUpdatedDataSource(),
    htmlFile = "../docs/batchCodes.html",
    lastUpdatedElementId = 'last_updated')

In [ ]:
updateBatchCodeTableHtmlFile(
    batchCodeTable,
    batchCodeTableHtmlFile = "../docs/HowBadIsMyBatch.html",
    lastUpdated = dateProvider.getLastUpdatedDataSource())

In [ ]:
symptomByBatchcodeTable = SymptomByBatchcodeTableFactory.createSymptomByBatchcodeTable(
    VAERSVAX = international_VAERSVAX_Covid19[international_VAERSVAX_Covid19['VAX_LOT'].isin(batchCodeTable['Batch'])],
    VAERSSYMPTOMS = international_VAERSSYMPTOMS)
symptomByBatchcodeTable

In [ ]:
createAndSaveGlobalHistograms(symptomByBatchcodeTable, batchCodeTable)

In [ ]:
countryCountsByBatchcode = filterByBatchcodes(getCountryCountsByBatchcodeTable(vaersBeforeDeletionDataDir), batchCodeTable['Batch'].values)
countryCountsByBatchcode

In [ ]:
barChartDescriptionTable = CountryCountsByBatchcodeTable2BarChartDescriptionTableConverter.convert2BarChartDescriptionTable(countryCountsByBatchcode)
barChartDescriptionTable

In [ ]:
barChartDescriptionTable = JensenShannonDistance2BarChartDescriptionColumnAdder.addJensenShannonDistance2BarChartDescriptionColumn(barChartDescriptionTable)
barChartDescriptionTable

In [ ]:
IOUtils.saveDictAsJson(
    BarChartDescriptionTable2DictionaryConverter.convert2Dictionary(barChartDescriptionTable, internationalVaersCovid19),
    '../docs/data/barChartDescriptionTable.json')


# Google Analytics

In [ ]:
cityCountsByClickedBatchcodeTable = CityCountsByBatchcodeTablesMerger.getCityCountsByClickedBatchcode('data/GoogleAnalytics')
cityCountsByClickedBatchcodeTable

In [ ]:
persistCityCountsByClickedBatchcodeTables(
    dataDir = Path('tmp/cityCountsByClickedBatchcodeTables'),
    n = 10,
    cityCountsByClickedBatchcodeTable = cityCountsByClickedBatchcodeTable)
# import generated xlsx files in dataDir into https://www.google.com/mymaps

# Vaccine Distribution by Zipcode

In [ ]:
vaccineDistributionByZipcode  = pd.read_excel(
        io = 'data/Amended-22-01962-Pfizer-2022-0426-pulled-2022-0823.xlsx',
        usecols = ['PROVIDER_NAME', 'ZIPCODE_SHP', 'LOT_NUMBER', 'DOSES_SHIPPED'])
vaccineDistributionByZipcode

In [ ]:
from VaccineDistributionByZipcodeSimplifier import VaccineDistributionByZipcodeSimplifier

vaccineDistributionByZipcode = VaccineDistributionByZipcodeSimplifier.sumDoses(vaccineDistributionByZipcode)
vaccineDistributionByZipcode

In [ ]:
OVERALL_DOSES_SHIPPED_by_LOT_NUMBER = (
    vaccineDistributionByZipcode
        .groupby('LOT_NUMBER')
        .agg(OVERALL_DOSES_SHIPPED = pd.NamedAgg(column = 'DOSES_SHIPPED', aggfunc = sum)))
OVERALL_DOSES_SHIPPED_by_LOT_NUMBER

In [ ]:
vaccineDistributionByZipcode = addColumn2Dataframe(dataframe = vaccineDistributionByZipcode, column = OVERALL_DOSES_SHIPPED_by_LOT_NUMBER)
vaccineDistributionByZipcode

In [ ]:
from ADR_by_Batchcode_Table_Factory import create_ADR_by_Batchcode_Table_4USA

ADR_by_Batchcode_Table_4USA = create_ADR_by_Batchcode_Table_4USA(internationalVaersCovid19)
ADR_by_Batchcode_Table_4USA

In [ ]:
vaccineDistributionByZipcode = addColumn2Dataframe(dataframe = vaccineDistributionByZipcode, column = ADR_by_Batchcode_Table_4USA)
vaccineDistributionByZipcode

In [ ]:
vaccineDistributionByZipcode['Statistical Number of Adverse Reaction Reports'] = (vaccineDistributionByZipcode['DOSES_SHIPPED'] / vaccineDistributionByZipcode['OVERALL_DOSES_SHIPPED'] * vaccineDistributionByZipcode['Adverse Reaction Reports']).round(2)
vaccineDistributionByZipcode['Statistical Number of Adverse Reaction Reports (per 100,000)'] = (vaccineDistributionByZipcode['DOSES_SHIPPED'] / vaccineDistributionByZipcode['OVERALL_DOSES_SHIPPED'] * 100000).round().astype(int)
vaccineDistributionByZipcode

In [ ]:
vaccineDistributionByZipcode = vaccineDistributionByZipcode[['PROVIDER_NAME', 'ZIPCODE_SHP', 'LOT_NUMBER', 'DOSES_SHIPPED', 'Statistical Number of Adverse Reaction Reports', 'Statistical Number of Adverse Reaction Reports (per 100,000)']]
vaccineDistributionByZipcode

In [ ]:
vaccineDistributionByZipcode = vaccineDistributionByZipcode.rename(
    columns = {
        'PROVIDER_NAME': 'Provider',
        'ZIPCODE_SHP': 'ZIP Code',
        'LOT_NUMBER': 'Lot Number',
        'DOSES_SHIPPED': 'Doses Shipped'
    })
vaccineDistributionByZipcode

In [ ]:
# vaccineDistributionByZipcode.to_excel('tmp/Amended-22-01962-Pfizer-2022-0426-pulled-2022-0823_sumDoses.xlsx')


In [ ]:
IOUtils.saveDataFrameAsJson(vaccineDistributionByZipcode, '../docs/data/vaccineDistributionByZipcode/VaccineDistributionByZipcode.json')
saveLastUpdated2HtmlFile(
    lastUpdated = dateProvider.getLastUpdatedDataSource(),
    htmlFile =  "../docs/VaccineDistributionByZipcode.html",
    lastUpdatedElementId = 'last_updated')

# Symptoms caused by Drugs (EudraVigilance)


In [ ]:
from SymptomsCausedByVaccines.DataFrameFilter import DataFrameFilter
from SymptomsCausedByVaccines.HtmlUpdater import updateHtmlFile, updateHtmlFile4SymptomsCausedByCOVIDLots
from SymptomsCausedByVaccines.PrrSeriesFactory import PrrSeriesFactory
from SymptomsCausedByVaccines.PrrSeriesTransformer import PrrSeriesTransformer
from SymptomsCausedByVaccines.ProportionalReportingRatiosPersister import saveProportionalReportingRatios
import os
import pandas as pd
import gc

In [ ]:
def saveProportionalReportingRatios4PrrByVaccineBySymptomWithoutZeroPrrs(prrByVaccineAndSymptom, webAppBaseDir):
    prrByVaccineBySymptom = PrrSeriesFactory.getPrrByVaccineBySymptom(prrByVaccineAndSymptom)

    prrByVaccineBySymptomWithoutZeroPrrs = PrrSeriesTransformer.filterByNonZeroPrrs(prrByVaccineBySymptom)
    del prrByVaccineBySymptom
    gc.collect()

    filenameBySymptom = saveProportionalReportingRatios(
        prrByVaccineBySymptomWithoutZeroPrrs,
        directory = os.path.normpath(webAppBaseDir + '/data/ProportionalReportingRatios/symptoms'))

    del prrByVaccineBySymptomWithoutZeroPrrs
    gc.collect()
    return filenameBySymptom

In [ ]:
def saveProportionalReportingRatios4PrrBySymptomByVaccineWithHighPrrs(prrByVaccineAndSymptom, webAppBaseDir):
    prrBySymptomByVaccine = PrrSeriesFactory.getPrrBySymptomByVaccine(prrByVaccineAndSymptom)
    prrBySymptomByVaccineWithHighPrrs = PrrSeriesTransformer.filterPrrs(prrBySymptomByVaccine, lambda prr: prr >= 2)
    del prrBySymptomByVaccine
    gc.collect()

    filenameByDrug = saveProportionalReportingRatios(
        prrBySymptomByVaccineWithHighPrrs,
        directory = os.path.normpath(webAppBaseDir + '/data/ProportionalReportingRatios/vaccines'))

    del prrBySymptomByVaccineWithHighPrrs
    gc.collect()
    return filenameByDrug

In [ ]:
def saveProportionalReportingRatiosAndUpdateHtmlFile(prrByVaccineAndSymptom, webAppBaseDir, defaultSelectVaccineOptionText):
    prrByVaccineAndSymptom = DataFrameFilter.withoutZeroRowsAndZeroColumns(prrByVaccineAndSymptom)
    filenameBySymptom = saveProportionalReportingRatios4PrrByVaccineBySymptomWithoutZeroPrrs(prrByVaccineAndSymptom, webAppBaseDir)
    filenameByDrug = saveProportionalReportingRatios4PrrBySymptomByVaccineWithHighPrrs(prrByVaccineAndSymptom, webAppBaseDir)
    updateHtmlFile(
        symptomsDescr = {
            'symptoms': list(prrByVaccineAndSymptom.columns),
            'filenameBySymptom': filenameBySymptom },
        vaccinesDescr = {
            'vaccines': list(prrByVaccineAndSymptom.index),
            'filenameByDrug': filenameByDrug },
        htmlFile = os.path.normpath(webAppBaseDir + '/index.html'),
        defaultSelectVaccineOptionText = defaultSelectVaccineOptionText)


In [ ]:
os.system('cd data; 7z x -y prrByDrugAndSymptom-EudraVigilance.7z')

In [ ]:
saveProportionalReportingRatiosAndUpdateHtmlFile(
    prrByVaccineAndSymptom = pd.read_csv(
        'data/prrByDrugAndSymptom-EudraVigilance.csv',
        index_col = 'DRUG'),
    webAppBaseDir = os.getcwd() + '/../docs/SymptomsCausedByDrugs',
    defaultSelectVaccineOptionText = 'Select Drug')

# Symptoms caused by Vaccines


In [ ]:
prrByVaccineAndSymptom = pd.read_csv(
    'data/tfiltered.csv',
    usecols = lambda x: x != "Unnamed: 0",
    index_col = 'VAX_TYPE')
prrByVaccineAndSymptom.index.name = 'VACCINE'
prrByVaccineAndSymptom

In [ ]:
saveProportionalReportingRatiosAndUpdateHtmlFile(
    prrByVaccineAndSymptom = prrByVaccineAndSymptom,
    webAppBaseDir = os.getcwd() + '/../docs/SymptomsCausedByVaccines',
    defaultSelectVaccineOptionText = 'Select Vaccine')

# Symptoms caused by COVID Lots

In [ ]:
prrByLotAndSymptom = pd.read_excel(
        io = 'data/symcolumns500.xlsx',
        index_col = 'VAX_LOT')
prrByLotAndSymptom


In [ ]:
prrByLotBySymptom = PrrSeriesFactory.getPrrByVaccineBySymptom(prrByLotAndSymptom)
prrByLotBySymptom

In [ ]:
prrBySymptomByLot = PrrSeriesFactory.getPrrBySymptomByVaccine(prrByLotAndSymptom)
prrBySymptomByLot

In [ ]:
prrByLotBySymptomWithoutZeroPrrs = PrrSeriesTransformer.filterByNonZeroPrrs(prrByLotBySymptom)
prrByLotBySymptomWithoutZeroPrrs

In [ ]:
prrBySymptomByLotWithHighPrrs = PrrSeriesTransformer.filterPrrs(prrBySymptomByLot, lambda prr: prr > 1)
prrBySymptomByLotWithHighPrrs

In [ ]:
webAppBaseDir = os.getcwd() + '/../docs/SymptomsCausedByCOVIDLots'

In [ ]:
saveProportionalReportingRatios(
    prrByLotBySymptomWithoutZeroPrrs,
    directory = os.path.normpath(webAppBaseDir + '/data/ProportionalReportingRatios/symptoms'))

In [ ]:
saveProportionalReportingRatios(
    prrBySymptomByLotWithHighPrrs,
    directory = os.path.normpath(webAppBaseDir + '/data/ProportionalReportingRatios/vaccines'))

In [ ]:
# FK-TODO: reactivate
#updateHtmlFile4SymptomsCausedByCOVIDLots(
#    symptoms = list(prrByLotAndSymptom.columns),
#    batches = list(prrByLotAndSymptom.index),
#    htmlFile = os.path.normpath(webAppBaseDir + '/index.html'))